# **Reaction Explorer**
* **This is an example of reaction explorer. Script might be different according to how you structured your data directories**

In [1]:
from thermochemistry import Thermochemistry
import json
from pathlib import Path

/home/key2tw/.local/bin/miniconda3/envs/project-with-ocpmodel/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* **Set vasp file located directory**

In [2]:
intermediate_src = Path().cwd().parent.joinpath('example/intermediates')
transitionstate_src = Path().cwd().parent.joinpath('example/transition_states')

In [3]:
def explorer(intermediate_src:str, transitionstate_src:str, temperature:list=[285]):

    thermo = Thermochemistry()
    
    # initiate bag of information
    bag_energy = {"intermediates": {}, "transition_states": {}}

    # explorer for intermediates
    subdirs_interm = [x for x in intermediate_src.iterdir() if x.is_dir()]

    for path in subdirs_interm:
        
        label = path.name
        bag_energy["intermediates"][label] = {"potential_energy": None, "free_energy": {}}
        
        energy_file = path.joinpath("OUTCAR")
        freq_file = path.joinpath("freq/freq.dat")
        
        pe = thermo.get_potential_energy(energy_file=energy_file, frequency_file=freq_file, cutoff=50, is_transition_state=False)
        bag_energy["intermediates"][label]["potential_energy"] = pe
        
        for t in temperature:
            ge = thermo.get_free_energy(energy_file=energy_file, frequency_file=freq_file, temperature=t, cutoff=50, is_transition_state=False)
            bag_energy["intermediates"][label]["free_energy"][t] = ge

    # explorer for transition states
    subdirs_ts = [x for x in transitionstate_src.iterdir() if x.is_dir()]

    for path in subdirs_ts:
        
        label = path.name
        bag_energy["transition_states"][label] = {"potential_energy": None, "free_energy": {}}
        
        energy_file = path.joinpath("OUTCAR")
        freq_file = path.joinpath("freq/freq.dat")
        
        pe = thermo.get_potential_energy(energy_file=energy_file, frequency_file=freq_file, cutoff=50, is_transition_state=True)
        bag_energy["transition_states"][label]["potential_energy"] = pe
        
        for t in temperature:
            ge = thermo.get_free_energy(energy_file=energy_file, frequency_file=freq_file, temperature=t, cutoff=50, is_transition_state=True)
            bag_energy["transition_states"][label]["free_energy"][t] = ge
            
    return bag_energy

In [4]:
bag = explorer(intermediate_src, transitionstate_src, temperature=[573.15])    

* **Save bag of energy information as json or insert to mongoDB for your convenience**\
In this example, we are going to convert the data to json file 

In [5]:
json_dir = Path().cwd().parent.joinpath('example/info_bag.json')

with open(json_dir, 'w') as f:
    json.dump(bag, f, indent=4)